In [ ]:
import numpy as np 
import pandas as pd 
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torch.nn.functional as F
import copy
from torch.nn.utils.rnn import pad_sequence
import torch.utils.data as data
import torch.nn.functional as F


# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
# Data statistic
def data_stat():
    separated_data = []
    labels = []

    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            if filename == "cross-era_annotations.csv" or filename=="testing.csv": continue

            path = os.path.join(dirname, filename)
            data_array = []
            df = pd.read_csv(path, header = None)
            for row in df.itertuples(index = False):
                if pd.notna(row[0]):
                        label = ((row[0].split('_'))[1].split('/'))[0]
                        if len(data_array)!= 0:
                            labels.append(label)
                            separated_data.append(data_array)
                            data_array=[]
                        else: continue
                else:
                    data_array.append(row[1:])
            labels.append(label)
            separated_data.append(data_array)


    lengths = [len(arr) for arr in separated_data]

    len_idx = []
    steps = 1000
    median_len = np.median(lengths)
    print(f"Median of the data: {median_len}")

    for i in range(len(lengths)):
        len_idx.append((lengths[i]//steps))

    for i in range((max(lengths)//steps)+1):
        occurence = len_idx.count(i)
        print(f"From length of {i*steps} to {(i+1)*steps}: "+ str(occurence))
        
    
    # Data preprocessing method 1 --> padding into median of data's length

    processed_data = []
    for data_arr in separated_data:
        if len(data_arr) >= median_len:
            processed_data.append(data_arr[:int(median_len)])
        else:
            padded_data = np.pad(data_arr, ((0, int(median_len)-len(data_arr)), (0,0)), mode='constant', constant_values=0)
            processed_data.append(padded_data)


    # print(len_idx)
    print(len(processed_data))
    print(len(labels))

In [2]:
# Data preprocessing method 2 --> Splitting data by self defined length


processed_data = []
labels = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename == "cross-era_annotations.csv" or filename == "testing.csv": continue
            
        path = os.path.join(dirname, filename)
        data_array = []
        df = pd.read_csv(path, header = None)
        for row in df.itertuples(index = False):
                    if pd.notna(row[0]):
                        label = ((row[0].split('_'))[1].split('/'))[0]
                        data_array=[]

                    elif len(data_array) == 100:
                        processed_data.append(data_array)
                        labels.append(label)
                        data_array = []
                    else: 
                        data_array.append(row[1:])
                        
                        

/tmp/ipykernel_34/2533281546.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header = None)


In [3]:
print(processed_data[1])

# transposed_data = np.roll(processed_data[0][5:10], 1,axis = 0)
# print(transposed_data)

[(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.8, 0.0995842, 0.237683, 0.000462333, 1.97254, 0.900086, 0.0, 0.310047, 0.0274636, 0.0880705, 3.72477, 0.352272, 0.0892524), (0.9, 0.0517734, 0.208381, 0.00231392, 0.627596, 0.758975, 0.383647, 0.00305628, 0.0104645, 0.0, 3.56806, 0.775066, 0.189792), (1.0, 0.000418317, 0.221696, 0.0552435, 0.402648, 1.58262, 0.11958, 0.00904782, 0.0418296, 0.283298, 4.32629, 0.230467, 0.000321715), (1.1, 0.000515347, 0.241635, 0.0, 0.134457, 1.90284, 0.0238204, 0.0444569, 0.0108937, 0.0204232, 4.28055, 0.141912, 0.000446513), (1.2, 0.0005

In [ ]:
label_map = {'romantic': 0, 'baroque': 1, 'modern': 2, 'addon': 3, 'classical': 4}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("Using Cuda")

class Dataset(data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, idx):
        # Get audio sequence
        input_data = np.array(self.data[idx])
        
#         input_data = input_data.reshape(-1, input_data.shape[-1])
        input_data = torch.tensor(input_data, dtype=torch.float32)

        label = [label_map[label_name] for label_name in self.labels]
        return input_data, torch.tensor(label[idx])

    def __len__(self):
        return len(self.data)


        

dataset = Dataset(data = processed_data, labels = labels)
train_length = int(len(dataset)*0.9)
test_length = int(len(dataset))-train_length

train_dataset,test_dataset=torch.utils.data.random_split(dataset,[train_length,test_length])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)



In [ ]:

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  

    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)  
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))  
        out = self.fc(out[:, -1, :])  
        prob = F.softmax(out, dim=1)  

        return out,prob

    
class CNN_BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(CNN_BiLSTM, self).__init__()
        # CNN layers
        self.cnn = nn.Sequential(
            nn.Conv1d(input_size, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(16, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Reshape for CNN (batch_size, input_size, sequence_length)
        batch_size, input_size, sequence_length = x.size()
        x = x.view(-1, input_size, sequence_length)  # Reshape to apply CNN independently to each feature sequence
        cnn_output = self.cnn(x)
        cnn_output = cnn_output.view(batch_size, sequence_length, -1)  # Reshape back to original size        
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)  
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(cnn_output, (h0, c0))  
        out = self.fc(out[:, -1, :])  
        prob = F.softmax(out, dim=1)  

        return out,prob

In [ ]:
def train_model(model, criterion,optimizer,train_loader, num_epochs=10):
    model.train() 
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # Forward
        outputs, probs = model(inputs)
        _, preds = torch.max(probs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        # Backward
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        correct += (preds == labels).sum().item()           

    epoch_acc = correct / total
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()      
    return model, optimizer, epoch_loss

In [ ]:
def val_model(model, criterion, optimizer, test_loader, num_epochs=10):
    model.eval() 
    running_loss = 0.0
    correct = 0
    total = 0
    y_pred=[]
    y_true=[]
    for inputs, labels in test_loader:   
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # Forward
        outputs,probs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(probs, 1)

        running_loss += loss.item() * inputs.size(0)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

        pred = outputs.data.argmax(1).cpu().numpy()
        y_pred.extend(pred)
        true = labels.data.cpu().numpy()
        y_true.extend(true)

    epoch_acc = correct / total
    epoch_loss = running_loss / len(test_loader.dataset)
    print(f'Testing Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()      
    return model, epoch_loss, epoch_acc, y_pred, y_true

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
def training_loop(model, criterion, optimizer, epochs, train_dataloader, test_dataloader):
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  # set objects for storing metrics
  train_loss = []
  val_loss = []
  y_trues=[]
  y_preds=[]
  # Train model
  early_stopper = EarlyStopper(patience=10, min_delta=0.1)

  for epoch in range(0, epochs):
    print(f'Epoch: {epoch+1}\t')
    # training
    model, optimizer, t_loss = train_model(model, criterion, optimizer, train_dataloader,num_epochs=epochs)
    train_loss.append(train_loss)

    # # validation
    with torch.no_grad():
      model, v_loss, acc, y_pred, y_true = val_model(model, criterion, optimizer, test_dataloader,num_epochs=epochs)
    
      val_loss.append(v_loss)
      y_trues.extend(y_true)
      y_preds.extend(y_pred)
    if early_stopper.early_stop(v_loss):
        print("Early Stopping...")
        break
    if acc > best_acc:
      best_acc = acc
      best_model_wts = copy.deepcopy(model.state_dict())

  print(f'Best val Acc: {best_acc:4f}')
  torch.save(best_model_wts, "./trained.pt")

  model.load_state_dict(best_model_wts)

  return train_loss,val_loss, y_trues, y_preds

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
if __name__ == '__main__': 

  model = CNN_BiLSTM(input_size=13,hidden_size=16,num_layers=2,num_classes=5).to(device)
  optimizer_SGD = torch.optim.SGD(model.parameters(), lr=0.05, momentum = 0.9)
  optimizer_adam = torch.optim.Adam(model.parameters(), lr=0.001)
  criterion = nn.CrossEntropyLoss()
  train_loss,val_loss, y_trues, y_preds = training_loop(model, criterion, optimizer_adam,750, train_loader, test_loader) 

In [ ]:
import matplotlib.pyplot as plt
 
plt.title("Validation loss of model")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.plot(val_loss, label='Validation Loss')
plt.show()